Это задание взято из конкурса на Kaggle https://www.kaggle.com/competitions/sberbank-russian-housing-market/overview

### Загружаю данные ###

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./train.csv')

In [3]:
df.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


__Выберем основные признаки, с которыми будем работать__

_price_doc_: цена продажи (это целевая переменная)\
_id_: идентификатор транзакции\
_timestamp_: дата транзакции\
_full_sq_: общая площадь в квадратных метрах, включая лоджии, балконы и другие нежилые помещения\
_life_sq_: жилая площадь в квадратных метрах без учета лоджий, балконов и других нежилых помещений\
_floor_: для квартир, этаж дома\
_max_floor_: количество этажей в здании\
_material_: материал стены\
_build_year_: год постройки\
_num_room_: количество жилых комнат\
_kitch_sq_: кухонная зона\
_state_: состояние квартиры\
_product_type_: покупка или инвестиции собственником-арендатором\
_sub_area_: название района

In [4]:
columns = ['price_doc', 'timestamp', 'full_sq', 'life_sq',
'floor', 'max_floor', 'material', 'build_year',
 'num_room', 'kitch_sq', 'state', 'product_type', 'sub_area']

df = df[columns]

In [5]:
df.head()

,price_doc,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area
0,5850000,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Bibirevo
1,6000000,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Nagatinskij Zaton
2,5700000,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Tekstil'shhiki
3,13100000,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Mitino
4,16331452,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Investment,Basmannoe


In [6]:
df.dtypes

price_doc         int64
timestamp        object
full_sq           int64
life_sq         float64
floor           float64
max_floor       float64
material        float64
build_year      float64
num_room        float64
kitch_sq        float64
state           float64
product_type     object
sub_area         object
dtype: object

### Обрабатываю пропущенные значения ###

Я заполню пропущенные значения самым частым значением в столбце.

In [7]:
from collections import Counter

def most_common_value(arr):
    counter = Counter(arr)
    most_common_value = counter.most_common(1)[0][0];
    return most_common_value

In [8]:
def fill_nan_values(columns):
    for col in columns:
        df[col].fillna(most_common_value(df[col]), inplace=True)

In [9]:
cols = ['life_sq', 'floor', 'max_floor', 'material', 'build_year', 'num_room', 'kitch_sq', 'state']
fill_nan_values(cols)

In [10]:
df.head()

,price_doc,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,sub_area
0,5850000,2011-08-20,43,27.0,4.0,17.0,1.0,2014.0,2.0,1.0,2.0,Investment,Bibirevo
1,6000000,2011-08-23,34,19.0,3.0,17.0,1.0,2014.0,2.0,1.0,2.0,Investment,Nagatinskij Zaton
2,5700000,2011-08-27,43,29.0,2.0,17.0,1.0,2014.0,2.0,1.0,2.0,Investment,Tekstil'shhiki
3,13100000,2011-09-01,89,50.0,9.0,17.0,1.0,2014.0,2.0,1.0,2.0,Investment,Mitino
4,16331452,2011-09-05,77,77.0,4.0,17.0,1.0,2014.0,2.0,1.0,2.0,Investment,Basmannoe


### Обрабатываю категориальные признаки ###

In [11]:
from sklearn.preprocessing import OneHotEncoder

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30471 entries, 0 to 30470
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price_doc     30471 non-null  int64  
 1   timestamp     30471 non-null  object 
 2   full_sq       30471 non-null  int64  
 3   life_sq       30471 non-null  float64
 4   floor         30471 non-null  float64
 5   max_floor     30471 non-null  float64
 6   material      30471 non-null  float64
 7   build_year    30471 non-null  float64
 8   num_room      30471 non-null  float64
 9   kitch_sq      30471 non-null  float64
 10  state         30471 non-null  float64
 11  product_type  30471 non-null  object 
 12  sub_area      30471 non-null  object 
dtypes: float64(8), int64(2), object(3)
memory usage: 3.0+ MB


In [13]:
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

encoder = OneHotEncoder(sparse_output=False)
one_hot_encoded = encoder.fit_transform(df[categorical_columns])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

one_hot_df.head()

,timestamp_2011-08-20,timestamp_2011-08-23,timestamp_2011-08-27,timestamp_2011-09-01,timestamp_2011-09-05,timestamp_2011-09-06,timestamp_2011-09-08,timestamp_2011-09-09,timestamp_2011-09-10,timestamp_2011-09-13,...,sub_area_Vnukovo,sub_area_Vojkovskoe,sub_area_Vostochnoe,sub_area_Vostochnoe Degunino,sub_area_Vostochnoe Izmajlovo,sub_area_Vyhino-Zhulebino,sub_area_Zamoskvorech'e,sub_area_Zapadnoe Degunino,sub_area_Zjablikovo,sub_area_Zjuzino
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
df2 = pd.concat([df, one_hot_df], axis=1)
df2 = df2.drop(categorical_columns, axis=1)
df2.head()

,price_doc,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,...,sub_area_Vnukovo,sub_area_Vojkovskoe,sub_area_Vostochnoe,sub_area_Vostochnoe Degunino,sub_area_Vostochnoe Izmajlovo,sub_area_Vyhino-Zhulebino,sub_area_Zamoskvorech'e,sub_area_Zapadnoe Degunino,sub_area_Zjablikovo,sub_area_Zjuzino
0,5850000,43,27.0,4.0,17.0,1.0,2014.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6000000,34,19.0,3.0,17.0,1.0,2014.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5700000,43,29.0,2.0,17.0,1.0,2014.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13100000,89,50.0,9.0,17.0,1.0,2014.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16331452,77,77.0,4.0,17.0,1.0,2014.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Откладываю часть выборки ###

In [16]:
df2_test = df2.sample(n=1000, random_state=0)
df2_train = df2.drop(index=df2_test.index)

Зададим объекты и целевое значение

In [17]:
X_train = df2_train[df2_train.columns[1:]]
y_train = df2_train['price_doc']

X_test = df2_test[df2_test.columns[1:]]
y_test = df2_test['price_doc']

### Метрика ###

Метрика, которую мне нужно будет улучшать это root mean squared logarithmic error (RMSLE). Она считается по формуле:
$$RMSLE = \sqrt{\frac{1}{n}\sum_{i=1}^n{(log(x_i + 1) - log(y_i + 1))^2}}$$

In [18]:
from sklearn.metrics import mean_squared_log_error
import numpy as np

Построю элементарный алгоритм

In [19]:
predictions = np.array([y_train.median()]*1000)

In [21]:
baseline = np.sqrt(mean_squared_log_error(y_test, predictions))

In [22]:
print(f"Значение метрики RMSLE при предскзаании медианой: {baseline}")

Значение метрики RMSLE при предскзаании медианой: 0.6043387231257158


### Предобработка данных ###

Я попробую два способа:\
_Cтандартизация_
$$x_i^j = \frac{x_i^j-\mu_j}{\sigma_j}$$ 
_Масштабирование на отрезке [0, 1]_
$$x_i^j = \frac{x_i^j-m_j}{M_j-m_j},$$ где $$m_j=min(x_1^j, x_2^j, ... , x_i^j),  M_j=max(x_1^j, x_2^j, ... , x_i^j)$$

In [23]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [24]:
min_max = MinMaxScaler()

min_max.fit(X_train)

X_train_min_max = min_max.transform(X_train)
X_test_min_max = min_max.transform(X_test)

In [25]:
standard = StandardScaler()

standard.fit(X_train)

X_train_standard = standard.transform(X_train)
X_test_standard = standard.transform(X_test)

### Подготовка к сравнению предсказаний ###

In [26]:
def calc_quality(X_test, X_train, y_test, y_train, model):
    predictions_test = model.predict(X_test)
    predictions_train = model.predict(X_train)

    if (predictions_test < 0).any() or (predictions_train < 0).any():
        predictions_test[np.where(predictions_test < 0)] = 0
        predictions_train[np.where(predictions_train < 0)] = 0

    test_quality = np.sqrt(mean_squared_log_error(y_test, predictions_test))
    train_quality = np.sqrt(mean_squared_log_error(y_train, predictions_train))

    print(f"Значение метрики RMSLE на тесте: {test_quality}")
    print(f"Значение метрики RMSLE на трейне: {train_quality}")


### Обучаю линейную регрессию с масштабированными данными ###

In [27]:
import sklearn.linear_model as lm

In [28]:
lin_reg = lm.LinearRegression().fit(X=X_train_min_max, y=y_train)

#### Сравниваю предсказания ####

In [29]:
calc_quality(X_test_min_max, X_train_min_max, y_test, y_train, lin_reg)

Значение метрики RMSLE на тесте: 0.5520527516626413
Значение метрики RMSLE на трейне: 0.645690254081272


### Обучаю линейную регрессию со стандартизированными данными ###

In [30]:
lin_reg_st = lm.LinearRegression().fit(X=X_train_standard, y=y_train)

#### Сравниваю предсказания ####

In [31]:
calc_quality(X_test_standard, X_train_standard, y_test, y_train, lin_reg_st)

Значение метрики RMSLE на тесте: 0.5494230776269143
Значение метрики RMSLE на трейне: 0.6199871368404444
